In [33]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from patchify import patchify

Images_DSdir = "Dataset/MassachusettsBuildings/Targets"
Targets_DSdir = "Dataset/MassachusettsBuildings/Targets"

In [34]:
Images_DS = []
ImageFiles = os.listdir(Images_DSdir)
PatchSize = 256
for ImageFile in ImageFiles:
    ImageCV = cv2.imread(Images_DSdir+'/'+ImageFile, cv2.IMREAD_COLOR)  # Read each image as BGR.
    ImageCV = cv2.cvtColor(ImageCV,cv2.COLOR_BGR2RGB)
    SizeX = (ImageCV.shape[1]//PatchSize)*PatchSize # Nearest size divisible by 256 (patch size).
    SizeY = (ImageCV.shape[0]//PatchSize)*PatchSize # Nearest size divisible by 256 (patch size).
    ImageCV = Image.fromarray(ImageCV)
    ImageCV = ImageCV.crop((0, 0, SizeX, SizeY))  # Crop from top left corner.
    ImageCV = np.array(ImageCV) 

    ImagePatches = patchify(ImageCV, (PatchSize, PatchSize, 3), step=PatchSize)  # Step = 256 for 256 patches means no overlap
    for i in range(ImagePatches.shape[0]):
        for j in range(ImagePatches.shape[1]):
            ImagePatch = ImagePatches[i,j,:,:]
            
            # #Use minmaxscaler instead of just dividing by 255. 
            # single_patch_img = scaler.fit_transform(single_patch_img.reshape(-1, single_patch_img.shape[-1])).reshape(single_patch_img.shape
            # # single_patch_img = (single_patch_img.astype('float32')) / 255. 

            ImagePatch = ImagePatch[0] #Drop the extra unecessary dimension that patchify adds.                               
            Images_DS.append(ImagePatch)

Images_DS = np.array(Images_DS)
print(Images_DS.shape)

(3750, 256, 256, 3)


In [42]:
Targets_DS = []
TargetFiles = os.listdir(Targets_DSdir)
PatchSize = 256
for TargetFile in TargetFiles:
    TargetCV = cv2.imread(Targets_DSdir+'/'+TargetFile, cv2.IMREAD_COLOR)  # Read each image as BGR.
    TargetCV = cv2.cvtColor(TargetCV,cv2.COLOR_BGR2RGB)
    SizeX = (TargetCV.shape[1]//PatchSize)*PatchSize # Nearest size divisible by 256 (patch size).
    SizeY = (TargetCV.shape[0]//PatchSize)*PatchSize # Nearest size divisible by 256 (patch size).
    TargetCV = Image.fromarray(TargetCV)
    TargetCV = TargetCV.crop((0, 0, SizeX, SizeY))  # Crop from top left corner.
    TargetCV = np.array(TargetCV) 

    TargetPatches = patchify(TargetCV, (PatchSize, PatchSize, 3), step=PatchSize)  # Step = 256 for 256 patches means no overlap
    for i in range(TargetPatches.shape[0]):
        for j in range(TargetPatches.shape[1]):
            TargetPatch = TargetPatches[i,j,:,:] # TargetPatches[i][j][:][:]
            
            # #Use minmaxscaler instead of just dividing by 255. 
            # single_patch_img = scaler.fit_transform(single_patch_img.reshape(-1, single_patch_img.shape[-1])).reshape(single_patch_img.shape
            TargetPatch = TargetPatch.astype("float32")/255.

            TargetPatch = TargetPatch[0] #Drop the extra unecessary dimension that patchify adds.                               
            Targets_DS.append(TargetPatch)

Targets_DS = np.array(Targets_DS)
print(Targets_DS.shape)

(3750, 256, 256, 3)


In [48]:
TargetLables = Targets_DS[:,:,:,0].astype("int")
TargetLables = np.array(TargetLables)
TargetLables = np.expand_dims(TargetLables, axis=3)
print(TargetLables.shape)

(3750, 256, 256, 1)


In [57]:
import torch
import torch.utils.data as data

Dataset = {"feature": torch.tensor([Images_DS], dtype=torch.float32), 
                "label": torch.tensor([TargetLables], dtype=torch.int)}
Train_DS, Test_DS = data.random_split(Dataset, [0.9, 0.1])